# Yellow Fever Epidemic Modeling: Senegal 2002 Outbreak 

This notebook demonstrates the application of a compartmental SEIR  model with vaccination intervention of the Yellow Fever outbreak in Touba, Senegal (population: 800,000). The data was sourced from WHO archives. 

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pathlib import Path 
import json 
import warnings 
warnings.filterwarnings('ignore')

import sys, os 
sys.path.append(os.path.abspath(os.path.join('..','src')))
from epimodels.yellow_fever_models import YellowFeverModel, create_vaccination_function

# set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Data Loading and Preparation

In [2]:
# Historical WHO outbreak data

data = {
    'date': ['2002-01-18', '2002-10-04', '2002-10-11', '2002-10-17', '2002-10-24', '2002-10-31', '2002-11-20', '2002-11-28'],
    'days_since_start': [0, 259, 266, 272, 279, 286, 306, 314],
    'cases_cumulative': [18, 12, 15, 18, 41, 45, 57, 60],
    'deaths_cumulative': [0, 0, 2, 2, 4, 4, 10, 11],
    'new_cases': [18, 12, 3, 3, 23, 4, 12, 3],
    'new_deaths': [0, 0, 2, 0, 2, 0, 6, 1]
}

data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])

print("Historical Data Overview:")
print(f"Number of observations: {len(data)}")
print(f"Time span: {data['days_since_start'].min()} to {data['days_since_start'].max()} days")
print(f"Date range: {data['date'].min().date()} to {data['date'].max().date()}")
print(f"Total cases: {data['cases_cumulative'].max()}")
print(f"Total deaths: {data['deaths_cumulative'].max()}")
print(f"Case Fatality Rate: {data['deaths_cumulative'].iloc[-1] / data['cases_cumulative'].iloc[-1] * 100:.1f}%")

print("\nComplete Dataset:")
display(data)

Historical Data Overview:
Number of observations: 8
Time span: 0 to 314 days
Date range: 2002-01-18 to 2002-11-28
Total cases: 60
Total deaths: 11
Case Fatality Rate: 18.3%

Complete Dataset:


,date,days_since_start,cases_cumulative,deaths_cumulative,new_cases,new_deaths
0,2002-01-18,0,18,0,18,0
1,2002-10-04,259,12,0,12,0
2,2002-10-11,266,15,2,3,2
3,2002-10-17,272,18,2,3,0
4,2002-10-24,279,41,4,23,2
5,2002-10-31,286,45,4,4,0
6,2002-11-20,306,57,10,12,6
7,2002-11-28,314,60,11,3,1


### Summary Statistics

In [3]:
summary_stats = {
    'Total Cases': data['cases_cumulative'].max(),
    'Total Deaths': data['deaths_cumulative'].max(),
    'Case Fatality Rate (%)': round(data['deaths_cumulative'].iloc[-1] / data['cases_cumulative'].iloc[-1] * 100, 2),
    'Peak Daily Cases': data['new_cases'].max(),
    'Peak Daily Deaths': data['new_deaths'].max(),
    'Observation Period (days)': data['days_since_start'].max(),
    'Attack Rate (%)': round(data['cases_cumulative'].max() / 800000 * 100, 4)
}

summary_df = pd.DataFrame(list(summary_stats.items()), columns=['Metric', 'Value'])
display(summary_df)

,Metric,Value
0,Total Cases,60.0000
1,Total Deaths,11.0000
2,Case Fatality Rate (%),18.3300
3,Peak Daily Cases,23.0000
4,Peak Daily Deaths,6.0000
5,Observation Period (days),314.0000
6,Attack Rate (%),0.0075


## Model Setup and Basic Simulation

### Model Structure

For this analysis, I use a compartmental SEIR model with vaccination: 

**Compartments**
- **S**: Susceptible individuals
- **E**: Exposed individuals (infected, but not yet infectious)
- **I**: Infectious individuals 
- **R**: Recovered individuals (natural immunity)
- **V**: Vaccinated (vaccine-induced immunity)
- **D**: Deaths (cumulative) 

**Parameters**
- $\beta$: Transmission rate 
- $\sigma$: Incubation period (days) 
- $\gamma$: Infectious period (days) 
- $\alpha$: Mortality rate 
### Differential Equations: 

$$\frac{dS}{dt} = -\beta \frac{SI}{N} - V(t)S$$
$$\frac{dE}{dt} = \beta \frac{SI}{N} - \sigma E$$
$$\frac{dI}{dt} = \sigma E - \gamma I - \alpha I$$
$$\frac{dR}{dt} = \gamma I $$
$$\frac{dV}{dt} = V(t)S $$
$$\frac{dD}{dt} = \alpha I $$





In [4]:
# vaccination function starting on October 1, 2002 (day 259)
vaccination_func = create_vaccination_function(
    start_date=259,
    vaccination_rate=0.01,       # 1% of susceptibles per day
    ramp_duration=7.0,
    vaccine_efficiency=0.95
)

# initialize model with example parameters
# Initialize model with example parameters
model_example = YellowFeverModel(
    beta=0.5,           # Transmission rate
    sigma=1/6,          # Incubation period ~6 days
    gamma=1/7,          # Infectious period ~7 days
    alpha=0.15/7,       # Mortality rate (CFR ~15%)
    vaccination_func=vaccination_func,
    population=800000
)

print(model_example)
print(f"\nWith these parameters, R₀ = {model_example.calculate_r0():.4f}")


YellowFeverModel(
  β=0.5000 (transmission rate)
  σ=0.1667 (1/incubation)
  γ=0.1429 (recovery rate)
  α=0.0214 (mortality rate)
  N=800000 (population)
  R₀=3.0435
)

With these parameters, R₀ = 3.0435
